# MISSING DATA

## OVERVIEW

- [Introduction](#INTRODUCTION)
- [Replacing Missing Data](#REPLACING-MISSING-DATA)  
    - [Finding Missing Data](#Finding-Missing-Data)  
    - [Replace Missing Data using the mean](#Replace-Missing-Data-using-the-mean)
- [EXAMPLE](#EXAMPLE---Imputing-Missing-Values-before-Building-an-Estimator)
- [Dropping Data](#DROPPING-DATA)


## INTRODUCTION

It's usual to find missing data when working with large datasets. In fact, you might not even find a single sample free of missing data. Annoying as this is, simply ignoring missing data usually isn't an option, as it can wreck havoc if not handled properly during your analysis. If not accounted for, missing data might lead you to erroneous conclusions about your samples by resulting in incorrect sums and means, and even by skewing distributions.

When data are missing in a variable of a particular case, it is very important to fill this variable with some intuitive data, if possible. Adding a reasonable estimate of a suitable data value for this variable is better than leaving it blank. The operation of deciding what data to use to fill these blanks is called **data imputation**.

If there is a strong pattern among the missing values of a variable (e.g. caused by a broken sensor), the variable should be eliminated from the model.

Basic methods for mitigating missing data:
* Any time missing data (nan) is encountered, **replace it with a scalar value** (mean, median, mode, etc). Imputation of missing values with the mean of the nonmissed cases is referred as __mean substitution__. (see note below)
* If working with time series and the data is ordered, **replace it with the immediate or previous non missing value**.
* Another method when dealing with time series is interpolating missing data with non-nan values that come immediately before and after (interpolating methods: nearest, cubic, spline, etc.)

<div class="alert alert-block alert-info" style="margin-top: 20px">
<strong>Note about MEAN SUBSTIUTION</strong>
<br/>
If some decision rule can be safely applied to supply a specific value to the missing data, it may be closer to the true value than even the mean substitution would be. For example, it is more reasonable to replace a missing value for number of children with zero rather than replace it with the mean or the median number of children based on all the other records (many couples are childless)
</div>

## REPLACING MISSING DATA

In [1]:
import pandas as pd

dataset = pd.read_csv('../data/00_DataPreparation/missing_data_example.csv')

In [2]:
dataset

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes
5,France,35.0,58000.0,Yes
6,Spain,NaN,52000.0,No
7,France,48.0,79000.0,Yes
8,Germany,50.0,83000.0,No
9,France,37.0,67000.0,Yes


In [3]:
dataset.dtypes

Country       object
Age          float64
Salary       float64
Purchased     object
dtype: object

### Finding Missing Data

In [4]:
dataset.describe()

,Age,Salary
count,9.000000,9.000000
mean,38.777778,63777.777778
std,7.693793,12265.579662
min,27.000000,48000.000000
25%,35.000000,54000.000000
50%,38.000000,61000.000000
75%,44.000000,72000.000000
max,50.000000,83000.000000


In [5]:
dataset.isnull()

,Country,Age,Salary,Purchased
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,True,False
5,False,False,False,False
6,False,True,False,False
7,False,False,False,False
8,False,False,False,False
9,False,False,False,False


In [6]:
dataset.isnull().sum()

Country      0
Age          1
Salary       1
Purchased    0
dtype: int64

An easy way to **show which columns have nans** in them

In [7]:
pd.isnull(dataset).any(axis=0)[pd.isnull(dataset).any(axis=0)==True]

Age       True
Salary    True
dtype: bool

An easy way to **show which rows have nans** in them

In [8]:
dataset[pd.isnull(dataset).any(axis=1)]

,Country,Age,Salary,Purchased
4,Germany,40.0,NaN,Yes
6,Spain,NaN,52000.0,No


### Replace Missing Data using the mean

In [9]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=True)
dataset[['Age', 'Salary']] = imputer.fit_transform(dataset[['Age', 'Salary']])
dataset

,Country,Age,Salary,Purchased
0,France,44.000000,72000.000000,No
1,Spain,27.000000,48000.000000,Yes
2,Germany,30.000000,54000.000000,No
3,Spain,38.000000,61000.000000,No
4,Germany,40.000000,63777.777778,Yes
5,France,35.000000,58000.000000,Yes
6,Spain,38.777778,52000.000000,No
7,France,48.000000,79000.000000,Yes
8,Germany,50.000000,83000.000000,No
9,France,37.000000,67000.000000,Yes


In [10]:
dataset.describe()

,Age,Salary
count,10.000000,10.000000
mean,38.777778,63777.777778
std,7.253777,11564.099406
min,27.000000,48000.000000
25%,35.500000,55000.000000
50%,38.388889,62388.888889
75%,43.000000,70750.000000
max,50.000000,83000.000000


## EXAMPLE - Imputing Missing Values before Building an Estimator

This example shows that imputing the missing values can give better results than discarding the samples containing any missing value. Imputing does not always improve the predictions, so please check via cross-validation. Sometimes dropping rows or using marker values is more effective.  

Missing values can be replaced by the mean, the median or the most frequent value using the strategy hyper-parameter. The median is a more robust estimator for data with high magnitude variables which could dominate results (otherwise known as a ‘long tail’).

In [11]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score

In [12]:
dataset = load_boston()
print(dataset['DESCR'])

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

In [13]:
dataset.keys()

dict_keys(['DESCR', 'data', 'target', 'feature_names'])

In [14]:
X = dataset['data']
X.shape

(506, 13)

In [15]:
y = dataset['target']
y.shape

(506,)

In [16]:
n_samples = X.shape[0]
n_features = X.shape[1]

In [17]:
df = pd.DataFrame(X, columns=dataset['feature_names'])
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [18]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.593761,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.596783,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.647423,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


In [19]:
df.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
dtype: int64

There's no missing data in the dataset

Let's estimate the score on the entire dataset, with no missing values

In [20]:
estimator = RandomForestRegressor(random_state=0, n_estimators=100)
score = cross_val_score(estimator, X, y) # by default a 3-fold cross validation is used

In [21]:
score

array([ 0.81091858,  0.56628405,  0.30857975])

In [22]:
print("Score with the entire dataset = {0:0.2f}".format(score.mean()))

Score with the entire dataset = 0.56


Let's add some missing values. Adding missing values in 75% of the lines

In [23]:
missing_rate = 0.75
n_missing_samples = np.floor(n_samples * missing_rate)

In [24]:
n_missing_samples

379.0

In [25]:
missing_samples = np.hstack((np.zeros(n_samples - n_missing_samples, dtype=np.bool),
                             np.ones(n_missing_samples, dtype=np.bool)))
missing_samples

/Volumes/MacintoshDATA/softdevelopmentbcn/miniconda2/envs/dspy3/lib/python3.5/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':
/Volumes/MacintoshDATA/softdevelopmentbcn/miniconda2/envs/dspy3/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)


array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True,

In [26]:
missing_samples.shape

(506,)

In [27]:
rng = np.random.RandomState(0)
rng.shuffle(missing_samples)
missing_samples

array([ True,  True,  True,  True, False, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True, False,  True, False,  True, False,
        True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True, False, False,
       False,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True,  True,
        True,  True,  True,  True, False, False, False,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True, False,
        True,  True,

In [28]:
missing_features = rng.randint(0, n_features, n_missing_samples)
missing_features

/Volumes/MacintoshDATA/softdevelopmentbcn/miniconda2/envs/dspy3/lib/python3.5/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':


array([ 8,  3,  6,  9, 10,  3,  2, 11,  7,  0,  3, 10,  0,  3,  6,  1, 12,
       11,  9,  2,  9,  4,  9, 11,  1,  3,  2,  4,  9,  7,  4,  9,  4,  1,
        2,  7,  2,  3,  9, 10,  7, 10,  6,  6, 12,  2, 10,  3,  6,  0, 12,
       10,  8,  0, 10, 11,  7,  6,  5,  9,  6,  5, 11,  2,  7, 12,  1,  9,
        2,  2, 12,  5, 11,  6,  4,  2, 12,  2,  1, 12,  0,  9,  0,  2,  8,
        3, 10,  0, 10, 11,  8,  8,  1,  0,  5,  8,  2,  3, 12,  5, 11,  3,
        8,  6,  4,  6, 12,  3,  6,  2, 12, 12,  6,  5, 11,  5,  9,  4,  6,
        5, 11,  1,  3,  3, 10,  8,  9,  5,  5,  6,  0,  9,  7,  5,  1,  5,
        6, 12, 10, 10, 11,  6,  8,  7,  5, 11, 10,  3, 10,  2,  9, 11,  9,
        3,  2,  5, 10,  4,  1,  5,  8,  3,  5,  8,  4, 10,  1,  7,  8,  1,
        2,  1,  1,  7,  5, 11,  0,  4,  1,  1, 12,  6,  6,  0,  2,  3,  7,
       12,  9,  2, 11,  4,  9,  0, 12,  6,  9,  2,  4,  7,  3,  0, 12,  5,
        4,  0,  2,  3,  1,  7, 10,  1,  3, 10,  4, 10,  1,  7,  4,  0, 10,
        2, 12, 10,  7,  4

In [29]:
missing_features.shape

(379,)

Estimate the **score without the lines containing missing values**

In [30]:
X_filtered = X[~missing_samples, :]
X_filtered.shape

(127, 13)

In [31]:
y_filtered = y[~missing_samples]
y_filtered.shape

(127,)

In [32]:
estimator = RandomForestRegressor(random_state=0, n_estimators=100)
score = cross_val_score(estimator, X_filtered, y_filtered)
print("Score with the entire dataset = {0:0.2f}".format(score.mean()))

Score with the entire dataset = 0.48


Estimate the **score after imputation of the missing values**

In [33]:
X_missing = X.copy()
X_missing[np.where(missing_samples)[0], missing_features]='nan'
X_missing

array([[  6.32000000e-03,   1.80000000e+01,   2.31000000e+00, ...,
          1.53000000e+01,   3.96900000e+02,   4.98000000e+00],
       [  2.73100000e-02,   0.00000000e+00,   7.07000000e+00, ...,
          1.78000000e+01,   3.96900000e+02,   9.14000000e+00],
       [  2.72900000e-02,   0.00000000e+00,   7.07000000e+00, ...,
          1.78000000e+01,   3.92830000e+02,   4.03000000e+00],
       ..., 
       [  6.07600000e-02,   0.00000000e+00,   1.19300000e+01, ...,
          2.10000000e+01,   3.96900000e+02,   5.64000000e+00],
       [  1.09590000e-01,   0.00000000e+00,   1.19300000e+01, ...,
          2.10000000e+01,   3.93450000e+02,   6.48000000e+00],
       [  4.74100000e-02,   0.00000000e+00,   1.19300000e+01, ...,
          2.10000000e+01,   3.96900000e+02,   7.88000000e+00]])

In [34]:
y_missing = y.copy()

In [35]:
df_missing = pd.DataFrame(X_missing, columns=dataset['feature_names'])
df_missing.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,NaN,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,NaN,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,NaN,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,NaN,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [36]:
estimator = Pipeline([("imputer", Imputer(missing_values='NaN', strategy='mean', axis=0)),
                      ("forest", RandomForestRegressor(random_state=0, n_estimators=100))])
score = cross_val_score(estimator, X_missing, y_missing)
print("Score with the entire dataset = {0:0.2f}".format(score.mean()))

Score with the entire dataset = 0.57


## DROPPING DATA

You should always first try to fill in missing data rather that deleting it. But if all else fails and you've given up on rectifying your nans, you can always remove the sample or column completely, so that it no longer negatively impacts your analysis. This should ever be used as a last resort!

In [37]:
df = df_missing.copy()
df.isnull().sum()

CRIM       29
ZN         26
INDUS      37
CHAS       33
NOX        28
RM         31
AGE        34
DIS        21
RAD        20
TAX        30
PTRATIO    33
B          22
LSTAT      35
dtype: int64

In [38]:
df[df['LSTAT'].isnull()]=0
df.isnull().sum()

CRIM       29
ZN         26
INDUS      37
CHAS       33
NOX        28
RM         31
AGE        34
DIS        21
RAD        20
TAX        30
PTRATIO    33
B          22
LSTAT       0
dtype: int64

In [39]:
removing_rows = df.dropna(axis=0)
removing_rows.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10
19,0.72580,0.0,8.14,0.0,0.538,5.727,69.5,3.7965,4.0,307.0,21.0,390.95,11.28
20,0.00000,0.0,0.00,0.0,0.000,0.000,0.0,0.0000,0.0,0.0,0.0,0.00,0.00


In [40]:
removing_columns = df.dropna(axis=1)
removing_columns.head()

,LSTAT
0,4.98
1,9.14
2,4.03
3,2.94
4,5.33
